In [ ]:
!pip install ollama pandas ta-lib numpy scipy aiohttp nest_asyncio yfinance

import ollama
import pandas as pd
import numpy as np
import yfinance as yf
import asyncio
import aiohttp
import json
from nest_asyncio import apply
apply()  # Required for Jupyter async support

class DataIngestionAgent:
    async def fetch(self, url, session, retries=3):
        for _ in range(retries):
            try:
                async with session.get(url, timeout=5) as response:
                    return await response.json()
            except (aiohttp.ClientError, asyncio.TimeoutError):
                await asyncio.sleep(2)
        return None

    async def gather_data(self, ticker):
        async with aiohttp.ClientSession() as session:
            tasks = [
                self.fetch(f"https://mock-news-api.com/latest?ticker={ticker}", session)
            ]
            results = await asyncio.gather(*tasks)

        # Fetch options data using yfinance
        stock = yf.Ticker(ticker)
        options_data = stock.option_chain()

        return {
            "options": options_data,
            "news": results[0] or [{"title": "Sample News", "content": "Sample content..."}]
        }

class OllamaClient:
    """Simplified async Ollama client using official library"""
    def __init__(self, model: str = "gemma:2b"):
        self.client = ollama.AsyncClient()
        self.model = model
        self.max_retries = 3

    async def generate_analysis(self, prompt: str) -> str:
        """Generate analysis with error handling"""
        for attempt in range(self.max_retries):
            try:
                response = await self.client.generate(
                    model=self.model,
                    prompt=prompt[:4000],  # Truncate to model context limit
                    options={'temperature': 0.5}
                )
                return response['response']
            except Exception as e:
                print(f"Attempt {attempt+1} failed: {e}")
                await asyncio.sleep(1)
        return "Error: Analysis unavailable"

class AnalysisAgent:
    def __init__(self):
        self.data_ingestor = DataIngestionAgent()
        self.llm_client = OllamaClient()

    async def analyze_ticker(self, ticker: str) -> dict:
        """Full analysis pipeline for a ticker"""
        # Gather market data
        data = await self.data_ingestor.gather_data(ticker)
        
        # Prepare LLM prompt
        analysis_prompt = f"""
        Perform financial analysis for {ticker} considering:
        1. Options Chain Data: {data['options'].calls[['strike', 'lastPrice']].head().to_dict()}
        2. Recent News Headlines: {[n['title'] for n in data['news'][:3]]}
        
        Provide technical analysis and trading recommendations in bullet points.
        """
        
        # Generate analysis
        analysis = await self.llm_client.generate_analysis(analysis_prompt)
        
        return {
            'ticker': ticker,
            'analysis': analysis,
            'timestamp': pd.Timestamp.now().isoformat(),
            'options_data': data['options'].calls[['strike', 'lastPrice']].head().to_dict(),
            'news': [n['title'] for n in data['news'][:3]]
        }

async def main():
    analyzer = AnalysisAgent()
    results = await analyzer.analyze_ticker("AAPL")
    print("Financial Analysis Report:")
    print(f"Ticker: {results['ticker']}")
    print(f"Timestamp: {results['timestamp']}")
    print("\nOptions Data Overview:")
    print(json.dumps(results['options_data'], indent=2))
    print("\nRecent News Headlines:")
    print("\n".join(results['news']))
    print("\nAI Analysis:")
    print(results['analysis'])

if __name__ == "__main__":
    asyncio.run(main())